# Practical Exam: Loan Insights

EasyLoan offers a wide range of loan services, including personal loans, car loans, and mortgages.

EasyLoan offers loans to clients from Canada, United Kingdom and United States.

The analytics team wants to report performance across different geographic areas. They aim to identify areas of strength and weakness for the business strategy team.

They need your help to ensure the data is accessible and reliable before they start reporting.


**Database Schema**

The data you need is in the database named `lending`.

![database schema](lending_schema.png)

# Task 1 

The analytics team wants to use the `client` table to create a dashboard for client details. For them to proceed, they need to be sure the data is clean enough to use.

The `client` table below illustrates what the analytics team expects the data types and format to be.

Write an SQL query that returns the `client` table with the specified format, including identifying and cleaning all invalid values. 
-  Your output should be a DataFrame with the name 'client'. Do not modify the `client` table.
-  Note that the DataLab environment formats dates as YYYY-MM-DD-hh-ss-SSS. 

| Column Name       | Description                                                      |
|-------------------|------------------------------------------------------------------|
| client_id         | Unique integer (set by the database, can’t take any other value) |
| date\_of\_birth       | Date of birth of the client, as a date                           |
| employment_status        | Current employment status of the client, either employed or unemployed, as a lower case string                              |
| country          | The country where the client resides, either USA, UK or CA, as an upper case string                      |

In [1]:
import pandas as pd
import sqlite3
db_path = '../database/dc_loandb.db3'

In [10]:
query = """
    SELECT  client_id, 
			CAST(date_of_birth AS DATE) date_of_birth, 
			LOWER(employment_status) AS employment_status, 
			UPPER(country) as country  
    FROM client;
    """
df_client = pd.read_sql_query(query, f"sqlite:///{db_path}")


In [12]:
df_client['employment_status'] = df_client['employment_status'].replace({
    'full-time': 'employed',
    'part-time': 'employed',
    'emplouyed': 'employed'
})



# Task 2

You have been told that there was a problem in the backend system as some of the `repayment_channel` values are missing. 

The missing values are critical to the analysis so they need to be filled in before proceeding.

Luckily, they have discovered a pattern in the missing values:

- Repayment higher than 4000 dollars should be made via `bank account`.
- Repayment lower than 1000 dollars should be made via `mail`.

Write an SQL query that makes the `repayment` table match these criteria.
-  Your output should be a DataFrame with the name 'repayment'. Do not modify the original `repayment` table.

In [ ]:
query = """
	SELECT repayment_id,
		loan_id,
		repayment_date,
		repayment_amount,
		repayment_channel	
		FROM repayment;
	"""
df_repayment = pd.read_sql_query(query, f"sqlite:///{db_path}")
df_repayment['repayment_channel'].value_counts()

In [ ]:
df_repayment.loc[(df_repayment['repayment_channel'] == '-') & (df_repayment['repayment_amount'] > 4000), 'repayment_channel'] = 'bank account'
df_repayment.loc[(df_repayment['repayment_channel'] == '-') & (df_repayment['repayment_amount'] < 1000), 'repayment_channel'] = 'mail'

df_repayment['repayment_channel'].value_counts()

# Task 3

Starting on January 1st, 2022, all US clients started to use an online system to sign contracts.

The analytics team wants to analyze the loans for US clients who used the new online system.

Write a query that returns the data for the analytics team. Your output should include `client_id`,`contract_date`, `principal_amount` and `loan_type` columns.

![database schema](lending_schema.png)

In [ ]:
query = """
    SELECT l.client_id, c.contract_date, l.principal_amount, l.loan_type
    FROM loan as l
    LEFT JOIN contract as c
    ON c.contract_id = l.contract_id
    LEFT JOIN client as cl
    ON cl.client_id = l.client_id
    WHERE c.contract_date>= '2022-01-01'
    AND cl.country = 'USA';
"""

df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df

# Task 4

The business strategy team is considering offering a more competitive rate to the US market. 

The analytic team want to compare the average interest rates offered by the company for the same loan type in different countries to determine if there are significant differences.

Write a query that returns the data for the analytics team. Your output should include `loan_type`, `country` and `avg_rate` columns.

![database schema](lending_schema.png)

In [ ]:
query = """
    SELECT l.loan_type, cl.country, 
        AVG(l.interest_rate) as avg_rate
    FROM loan as l    
    LEFT JOIN client as cl
    ON cl.client_id = l.client_id
    GROUP BY l.loan_type, cl.country
    ORDER BY l.loan_type, cl.country;
"""

df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df